# Section 3: Real-time Token Updates

In this section, you'll learn to:
- Create interactive OHLCV charts for price analysis
- Implement real-time price monitoring with WebSocket
- Create live transaction feeds
- Monitor OHLCV data updates in real-time
- Track token statistics with advanced filters

## APIs & WebSocket Subscriptions Used:

### REST API Endpoints:
- **`GET /defi/v3/ohlcv`** - Fetch OHLCV (candlestick) data for charting
- **`GET /defi/price`** - Get current token prices

### WebSocket Subscriptions:
- **`SUBSCRIBE_PRICE`** - Real-time price updates for multiple tokens
- **`SUBSCRIBE_TXS`** - Live transaction feeds for specific tokens
- **`SUBSCRIBE_OHLCV`** - Real-time OHLCV candlestick updates
- **`SUBSCRIBE_TOKEN_STATS`** - Token statistics with filtering capabilities

### Response Fields You'll Work With:
- **OHLCV Data**: `items[]` with `o`, `h`, `l`, `c`, `v`, `unix_time`
- **WebSocket Price**: `address`, `value`, `updateUnixTime`
- **WebSocket Transactions**: `txType`, `amount`, `price`, `txHash`, `timestamp`
- **WebSocket OHLCV**: `o`, `h`, `l`, `c`, `v`, `unix_time`
- **WebSocket Token Stats**: `symbol`, `price`, `priceChange24h`, `volume24h`, `mc`, `liquidity`

**Reference**: [Token Stats WebSocket Documentation](https://docs.birdeye.so/docs/subscribe_token_stats#/)

Let's build real-time monitoring systems for blockchain data!

## Setup and Imports

**Important**: This section requires a **Business API Key** from Birdeye Data Services for WebSocket features.

### Getting Your Business API Key

The real-time WebSocket features require access to premium endpoints that are only available with a Business package. Your workshop host will provide you with a Business API key.

**Add the Business API key to your .env file:**
```
BDS_API_KEY=your_business_api_key_here
```

If you don't have a Business API key, you can:
1. Contact your workshop host for a temporary key
2. Sign up for a Business plan at [Birdeye Data Services](https://bds.birdeye.so)
3. Continue with the demo to see how the features work

In [ ]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets
from datetime import datetime, timedelta
import json
import time
import threading
import warnings
warnings.filterwarnings('ignore')

# Force reload of utils module to ensure latest version
import utils
import importlib
importlib.reload(utils)

# Import our custom utilities
from utils import (
    BirdeyeDataServices, 
    BirdeyeDataServicesWebSocket,
    create_portfolio_chart,
    create_portfolio_pie_chart,
    create_candlestick_chart,
    format_currency,
    format_transaction_data,
    check_api_key
)

print("All modules imported successfully for real-time updates!")

# Check Business API key and initialize client
if check_api_key('business'):
    # Initialize Birdeye Data Services client with Business API key
    birdeye = BirdeyeDataServices(api_key_type='business')
    print("Setup complete! Ready for real-time blockchain data monitoring.")
else:
    print("Please configure BDS_API_KEY in your .env file")
    print("Contact your workshop host for a Business API key")

### Troubleshooting Note

**If you encounter import errors:**
1. **Restart the kernel**: Go to `Kernel` → `Restart` in the menu
2. **Re-run the import cell above**
3. **Clear browser cache** if the issue persists

The functions exist in `utils.py` and the reload code above should resolve any caching issues.

## Step 1: Interactive OHLCV Charts

Let's start with interactive OHLCV charts for detailed price analysis:

In [ ]:
# Step 1: Interactive OHLCV Charts
def create_interactive_ohlcv_chart(token_address=None, token_symbol="Token"):
    if not token_address:
        token_address = "So11111111111111111111111111111111111111112"  # Default to SOL
        token_symbol = "SOL"
    
    print(f"Creating OHLCV chart for {token_symbol}...")
    
    ohlcv_data = birdeye.get_ohlcv_data(token_address, type_="1D")
    
    if ohlcv_data:
        candlestick_chart = create_candlestick_chart(ohlcv_data, token_symbol)
        
        if candlestick_chart:
            print(f"\n{token_symbol} OHLCV Chart:")
            candlestick_chart.show()
            
            print("\nChart Features:")
            print("   Green candles: Price increased (Close > Open)")
            print("   Red candles: Price decreased (Close < Open)")
            print("   Volume bars: Trading activity")
            print("   Interactive: Zoom, pan, and hover for details")
        else:
            print("Could not create chart")
    else:
        print("Could not fetch OHLCV data")

# Interactive controls for Step 1
step1_checkbox = widgets.Checkbox(
    value=False,
    description='Use custom token address',
    style={'description_width': 'initial'}
)

step1_token_input = widgets.Text(
    value='',
    placeholder='Enter token address',
    description='Token:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

step1_symbol_input = widgets.Text(
    value='',
    placeholder='Enter token symbol (e.g., USDC)',
    description='Symbol:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

step1_button = widgets.Button(
    description='Create OHLCV Chart',
    button_style='success',
    tooltip='Create candlestick chart'
)

step1_output = widgets.Output()

def on_step1_button_click(b):
    with step1_output:
        clear_output(wait=True)
        if step1_checkbox.value and step1_token_input.value.strip():
            symbol = step1_symbol_input.value.strip() or "Token"
            create_interactive_ohlcv_chart(step1_token_input.value.strip(), symbol)
        else:
            create_interactive_ohlcv_chart()

step1_button.on_click(on_step1_button_click)

# Initial execution
create_interactive_ohlcv_chart()

# Display interactive controls
print("\n" + "="*60)
print("Interactive Controls:")
display(widgets.VBox([
    step1_checkbox,
    widgets.HBox([step1_token_input, step1_symbol_input]),
    step1_button,
    step1_output
]))

## Step 2: Real-time OHLCV Updates

Let's implement real-time OHLCV updates using WebSocket connections:

In [ ]:
# Real-time price monitoring with WebSocket
class RealTimePriceMonitor:
    def __init__(self):
        self.ws = None
        self.price_data = {}
        self.is_running = False
        
    def start_monitoring(self, token_addresses):
        """Start real-time price monitoring"""
        if self.is_running:
            print("Monitoring already running")
            return
            
        print(f"Starting price monitoring for {len(token_addresses)} tokens...")
        self.is_running = True
        
        try:
            # Initialize WebSocket
            self.ws = BirdeyeDataServicesWebSocket()
            
            # Set up price update callback
            def on_price_update(data):
                try:
                    if 'data' in data:
                        address = data['data'].get('address')
                        price = data['data'].get('value')
                        
                        if address and price:
                            self.price_data[address] = {
                                'price': float(price),
                                'timestamp': datetime.now()
                            }
                            
                            # Print directly for better WebSocket output visibility
                            print(f"\nReal-time Price Updates:")
                            print(f"{'='*50}")
                            
                            for addr, info in self.price_data.items():
                                symbol = 'SOL' if addr == 'So11111111111111111111111111111111111111112' else addr[:8]
                                timestamp = info['timestamp'].strftime('%H:%M:%S')
                                print(f"{symbol}: ${info['price']:.6f} (Updated: {timestamp})")
                            print()  # Add spacing
                except Exception as e:
                    print(f"Error processing price update: {e}")
            
            # Use the subscribe_price method with callback
            def price_callback(data):
                on_price_update(data)
            
            # Connect and subscribe
            self.ws.connect()
            print("WebSocket connected successfully")
            
            for address in token_addresses:
                self.ws.subscribe_price(address, price_callback)
                print(f"Subscribed to price updates for {address[:8]}...")
                
        except Exception as e:
            print(f"WebSocket connection failed: {e}")
            print(f"Error details: {type(e).__name__}")
            self.is_running = False
    
    def stop_monitoring(self):
        """Stop real-time monitoring"""
        if self.ws:
            self.ws.close()
        self.is_running = False
        print("Real-time monitoring stopped")

# Create price monitor instance
price_monitor = RealTimePriceMonitor()

# Control buttons
start_button = widgets.Button(
    description='Start Real-time Monitoring',
    button_style='success',
    tooltip='Start WebSocket price monitoring'
)

stop_button = widgets.Button(
    description='Stop Monitoring',
    button_style='danger',
    tooltip='Stop WebSocket monitoring'
)

def on_start_click(b):
    # Monitor SOL and USDC prices
    tokens_to_monitor = [
        "So11111111111111111111111111111111111111112",  # SOL
        "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"   # USDC
    ]
    price_monitor.start_monitoring(tokens_to_monitor)

def on_stop_click(b):
    price_monitor.stop_monitoring()

start_button.on_click(on_start_click)
stop_button.on_click(on_stop_click)

print("Real-time Price Monitoring:")
display(widgets.HBox([start_button, stop_button]))

# Interactive controls for Step 2
step2_checkbox = widgets.Checkbox(
    value=False,
    description='Use custom token addresses',
    style={'description_width': 'initial'}
)

step2_tokens_input = widgets.Textarea(
    value='',
    placeholder='Enter token addresses (one per line)',
    description='Tokens:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px', height='100px')
)

step2_start_button = widgets.Button(
    description='Start Custom Monitoring',
    button_style='success',
    tooltip='Start WebSocket monitoring with custom tokens'
)

def on_step2_start_click(b):
    if step2_checkbox.value and step2_tokens_input.value.strip():
        token_addresses = [addr.strip() for addr in step2_tokens_input.value.strip().split('\n') if addr.strip()]
        if token_addresses:
            price_monitor.start_monitoring(token_addresses)
        else:
            print("Please enter at least one token address")
    else:
        # Default tokens
        default_tokens = [
            "So11111111111111111111111111111111111111112",  # SOL
            "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"   # USDC
        ]
        price_monitor.start_monitoring(default_tokens)

step2_start_button.on_click(on_step2_start_click)

print("\n" + "="*60)
print("Interactive Controls:")
display(widgets.VBox([
    step2_checkbox,
    step2_tokens_input,
    step2_start_button
]))

print("\nWebSocket Features:")
print("   Real-time price updates")
print("   Low latency data")
print("   Subscribe to specific tokens")
print("   Connection error handling")

## Step 3: Live Transaction Feed

Let's create a live transaction feed for monitoring token activity:

In [ ]:
# Live transaction monitoring
class TransactionMonitor:
    def __init__(self):
        self.ws = None
        self.transactions = []
        self.is_running = False
        
    def start_monitoring(self, token_address):
        """Start monitoring transactions for a token"""
        if self.is_running:
            print("Transaction monitoring already running")
            return
            
        print(f"Starting transaction monitoring for {token_address[:8]}...")
        self.is_running = True
        self.transactions = []
        
        try:
            # Initialize WebSocket
            self.ws = BirdeyeDataServicesWebSocket()
            
            # Set up transaction callback
            def on_transaction(data):
                try:
                    if 'data' in data:
                        tx_data = data['data']
                        self.transactions.append({
                            'timestamp': datetime.now(),
                            'type': tx_data.get('txType', 'Unknown'),
                            'amount': tx_data.get('amount', 0),
                            'price': tx_data.get('price', 0),
                            'signature': tx_data.get('txHash', '')[:10] + '...'
                        })
                        
                        # Keep only last 10 transactions
                        if len(self.transactions) > 10:
                            self.transactions = self.transactions[-10:]
                        
                        # Print directly for better WebSocket output visibility
                        print(f"\nLive Transaction Feed:")
                        print(f"{'='*60}")
                        
                        for tx in reversed(self.transactions):  # Show newest first
                            timestamp = tx['timestamp'].strftime('%H:%M:%S')
                            tx_type = tx['type']
                            amount = tx['amount']
                            price = tx['price']
                            signature = tx['signature']
                            
                            print(f"[{timestamp}] {tx_type}: {amount:.4f} @ ${price:.6f} ({signature})")
                        print()  # Add spacing
                except Exception as e:
                    print(f"Error processing transaction update: {e}")
            
            # Use the subscribe_transactions method with callback
            def transaction_callback(data):
                on_transaction(data)
            
            # Connect and subscribe
            self.ws.connect()
            print("WebSocket connected successfully")
            
            self.ws.subscribe_transactions(token_address, transaction_callback)
            print(f"Subscribed to transactions for {token_address[:8]}...")
            
        except Exception as e:
            print(f"WebSocket connection failed: {e}")
            print(f"Error details: {type(e).__name__}")
            self.is_running = False
    
    def stop_monitoring(self):
        """Stop transaction monitoring"""
        if self.ws:
            self.ws.close()
        self.is_running = False
        print("Transaction monitoring stopped")

# Create transaction monitor
tx_monitor = TransactionMonitor()

# Control buttons for transaction monitoring
start_tx_button = widgets.Button(
    description='Start Transaction Feed',
    button_style='info',
    tooltip='Start monitoring live transactions'
)

stop_tx_button = widgets.Button(
    description='Stop Transaction Feed',
    button_style='warning',
    tooltip='Stop transaction monitoring'
)

def on_start_tx_click(b):
    # Monitor SOL transactions
    sol_address = "So11111111111111111111111111111111111111112"
    tx_monitor.start_monitoring(sol_address)

def on_stop_tx_click(b):
    tx_monitor.stop_monitoring()

start_tx_button.on_click(on_start_tx_click)
stop_tx_button.on_click(on_stop_tx_click)

print("Live Transaction Monitoring:")
display(widgets.HBox([start_tx_button, stop_tx_button]))

# Interactive controls for Step 3
step3_checkbox = widgets.Checkbox(
    value=False,
    description='Use custom token address',
    style={'description_width': 'initial'}
)

step3_token_input = widgets.Text(
    value='',
    placeholder='Enter token address for transaction monitoring',
    description='Token:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

step3_start_button = widgets.Button(
    description='Start Custom TX Monitoring',
    button_style='info',
    tooltip='Start transaction monitoring for custom token'
)

def on_step3_start_click(b):
    if step3_checkbox.value and step3_token_input.value.strip():
        tx_monitor.start_monitoring(step3_token_input.value.strip())
    else:
        # Default to SOL
        sol_address = "So11111111111111111111111111111111111111112"
        tx_monitor.start_monitoring(sol_address)

step3_start_button.on_click(on_step3_start_click)

print("\n" + "="*60)
print("Interactive Controls:")
display(widgets.VBox([
    step3_checkbox,
    step3_token_input,
    step3_start_button
]))

print("\nTransaction Feed Features:")
print("   Real-time transaction updates")
print("   Buy/Sell activity tracking")
print("   Price and volume information")
print("   Transaction signature links")

In [ ]:
# Real-time OHLCV monitoring with WebSocket
class OHLCVMonitor:
    def __init__(self):
        self.ws = None
        self.ohlcv_data = {}
        self.is_running = False
        
    def start_monitoring(self, token_address, timeframe='1m'):
        """Start real-time OHLCV monitoring"""
        if self.is_running:
            print("OHLCV monitoring already running")
            return
            
        print(f"Starting OHLCV monitoring for {token_address[:8]}... (timeframe: {timeframe})")
        self.is_running = True
        
        try:
            # Initialize WebSocket
            self.ws = BirdeyeDataServicesWebSocket()
            
            # Set up OHLCV update callback
            def on_ohlcv_update(data):
                try:
                    if 'data' in data:
                        ohlcv_info = data['data']
                        
                        self.ohlcv_data[token_address] = {
                            'open': ohlcv_info.get('o', 0),
                            'high': ohlcv_info.get('h', 0),
                            'low': ohlcv_info.get('l', 0),
                            'close': ohlcv_info.get('c', 0),
                            'volume': ohlcv_info.get('v', 0),
                            'timestamp': datetime.now()
                        }
                        
                        # Print OHLCV updates
                        print(f"\nReal-time OHLCV Updates:")
                        print(f"{'='*50}")
                        
                        info = self.ohlcv_data[token_address]
                        timestamp = info['timestamp'].strftime('%H:%M:%S')
                        print(f"Token: {token_address[:8]}... (Updated: {timestamp})")
                        print(f"  Open: ${info['open']:.6f}")
                        print(f"  High: ${info['high']:.6f}")
                        print(f"  Low: ${info['low']:.6f}")
                        print(f"  Close: ${info['close']:.6f}")
                        print(f"  Volume: {info['volume']:.2f}")
                        print()
                except Exception as e:
                    print(f"Error processing OHLCV update: {e}")
            
            # Note: OHLCV WebSocket subscription not available in current implementation
            # This is a demonstration of how it would work
            print("Note: Real-time OHLCV updates require advanced WebSocket subscription")
            print("This demo shows the interface - actual implementation would need:")
            print(f"- WebSocket subscription to OHLCV data for {token_address[:8]}...")
            print(f"- Timeframe: {timeframe}")
            
            # Simulate some OHLCV data for demonstration
            import random
            base_price = 240.0
            
            def simulate_ohlcv():
                nonlocal base_price
                # Simulate price movement
                change = random.uniform(-0.02, 0.02)  # ±2% change
                base_price *= (1 + change)
                
                simulated_data = {
                    'data': {
                        'o': base_price * 0.998,  # Open slightly lower
                        'h': base_price * 1.005,  # High slightly higher
                        'l': base_price * 0.995,  # Low slightly lower
                        'c': base_price,          # Close at current price
                        'v': random.uniform(1000, 5000)  # Random volume
                    }
                }
                on_ohlcv_update(simulated_data)
            
            # Simulate periodic updates
            print("\nStarting simulated OHLCV updates...")
            for i in range(3):  # Show 3 updates
                simulate_ohlcv()
                if i < 2:  # Don't sleep after last update
                    time.sleep(2)
                
        except Exception as e:
            print(f"WebSocket connection failed: {e}")
            print(f"Error details: {type(e).__name__}")
            self.is_running = False
    
    def stop_monitoring(self):
        """Stop OHLCV monitoring"""
        if self.ws:
            self.ws.close()
        self.is_running = False
        print("OHLCV monitoring stopped")

# Create OHLCV monitor instance
ohlcv_monitor = OHLCVMonitor()

# Interactive controls for OHLCV monitoring
ohlcv_checkbox = widgets.Checkbox(
    value=False,
    description='Use custom token address',
    style={'description_width': 'initial'}
)

ohlcv_token_input = widgets.Text(
    value='',
    placeholder='Enter token address for OHLCV monitoring',
    description='Token:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

ohlcv_timeframe_select = widgets.Dropdown(
    options=['1m', '5m', '15m', '1h', '4h', '1d'],
    value='1m',
    description='Timeframe:',
    style={'description_width': 'initial'}
)

ohlcv_start_button = widgets.Button(
    description='Start OHLCV Monitoring',
    button_style='success',
    tooltip='Start real-time OHLCV monitoring'
)

ohlcv_stop_button = widgets.Button(
    description='Stop OHLCV Monitoring',
    button_style='danger',
    tooltip='Stop OHLCV monitoring'
)

def on_ohlcv_start_click(b):
    if ohlcv_checkbox.value and ohlcv_token_input.value.strip():
        ohlcv_monitor.start_monitoring(ohlcv_token_input.value.strip(), ohlcv_timeframe_select.value)
    else:
        # Default to SOL
        sol_address = "So11111111111111111111111111111111111111112"
        ohlcv_monitor.start_monitoring(sol_address, ohlcv_timeframe_select.value)

def on_ohlcv_stop_click(b):
    ohlcv_monitor.stop_monitoring()

ohlcv_start_button.on_click(on_ohlcv_start_click)
ohlcv_stop_button.on_click(on_ohlcv_stop_click)

print("Real-time OHLCV Monitoring:")
display(widgets.VBox([
    ohlcv_checkbox,
    ohlcv_token_input,
    ohlcv_timeframe_select,
    widgets.HBox([ohlcv_start_button, ohlcv_stop_button])
]))

print("\nOHLCV Features:")
print("   Real-time candlestick updates")
print("   Multiple timeframe support")
print("   Volume tracking")
print("   Live price action monitoring")

## Step 4: Token Statistics with Filters

Let's implement real-time token statistics monitoring with filtering capabilities:

In [ ]:
# Real-time token statistics monitoring with filters
class TokenStatsMonitor:
    def __init__(self):
        self.ws = None
        self.stats_data = {}
        self.is_running = False
        
    def start_monitoring(self, filters=None):
        """Start real-time token statistics monitoring with filters"""
        if self.is_running:
            print("Token stats monitoring already running")
            return
            
        print(f"Starting token statistics monitoring with filters...")
        self.is_running = True
        
        try:
            # Initialize WebSocket
            self.ws = BirdeyeDataServicesWebSocket()
            
            # Set up token stats update callback
            def on_token_stats_update(data):
                try:
                    if 'data' in data:
                        stats_info = data['data']
                        token_address = stats_info.get('address', 'unknown')
                        
                        self.stats_data[token_address] = {
                            'symbol': stats_info.get('symbol', 'Unknown'),
                            'price': stats_info.get('price', 0),
                            'price_change_24h': stats_info.get('priceChange24h', 0),
                            'volume_24h': stats_info.get('volume24h', 0),
                            'market_cap': stats_info.get('mc', 0),
                            'liquidity': stats_info.get('liquidity', 0),
                            'timestamp': datetime.now()
                        }
                        
                        # Print token stats updates
                        print(f"\nReal-time Token Statistics:")
                        print(f"{'='*60}")
                        
                        for addr, info in self.stats_data.items():
                            timestamp = info['timestamp'].strftime('%H:%M:%S')
                            price_change = info['price_change_24h']
                            change_symbol = '+' if price_change >= 0 else ''
                            
                            print(f"Token: {info['symbol']} ({addr[:8]}...) - Updated: {timestamp}")
                            print(f"  Price: ${info['price']:.6f} ({change_symbol}{price_change:.2f}%)")
                            print(f"  Volume 24h: ${info['volume_24h']:,.2f}")
                            print(f"  Market Cap: ${info['market_cap']:,.2f}")
                            print(f"  Liquidity: ${info['liquidity']:,.2f}")
                            print()
                except Exception as e:
                    print(f"Error processing token stats update: {e}")
            
            # Note: Token Stats WebSocket subscription not available in current implementation
            # This is a demonstration of how it would work
            print("Note: Real-time token statistics require advanced WebSocket subscription")
            print("This demo shows the interface - actual implementation would need:")
            print(f"- WebSocket subscription to token statistics")
            print(f"- Filters: {filters or 'None'}")
            print("- Reference: https://docs.birdeye.so/docs/subscribe_token_stats#/")
            
            # Simulate some token stats data for demonstration
            import random
            
            def simulate_token_stats():
                # Simulate data for SOL
                simulated_data = {
                    'data': {
                        'address': 'So11111111111111111111111111111111111111112',
                        'symbol': 'SOL',
                        'price': random.uniform(230, 250),
                        'priceChange24h': random.uniform(-5, 5),
                        'volume24h': random.uniform(50000000, 100000000),
                        'mc': random.uniform(100000000000, 120000000000),
                        'liquidity': random.uniform(10000000, 20000000)
                    }
                }
                on_token_stats_update(simulated_data)
            
            # Simulate periodic updates
            print("\nStarting simulated token statistics updates...")
            for i in range(3):  # Show 3 updates
                simulate_token_stats()
                if i < 2:  # Don't sleep after last update
                    time.sleep(2)
                
        except Exception as e:
            print(f"WebSocket connection failed: {e}")
            print(f"Error details: {type(e).__name__}")
            self.is_running = False
    
    def stop_monitoring(self):
        """Stop token stats monitoring"""
        if self.ws:
            self.ws.close()
        self.is_running = False
        print("Token stats monitoring stopped")

# Create token stats monitor instance
token_stats_monitor = TokenStatsMonitor()

# Interactive controls for token stats monitoring
stats_min_volume_input = widgets.FloatText(
    value=10000,
    description='Min Volume 24h:',
    style={'description_width': 'initial'}
)

stats_min_mc_input = widgets.FloatText(
    value=1000000,
    description='Min Market Cap:',
    style={'description_width': 'initial'}
)

stats_min_liquidity_input = widgets.FloatText(
    value=50000,
    description='Min Liquidity:',
    style={'description_width': 'initial'}
)

stats_start_button = widgets.Button(
    description='Start Token Stats Monitoring',
    button_style='info',
    tooltip='Start real-time token statistics monitoring'
)

stats_stop_button = widgets.Button(
    description='Stop Token Stats Monitoring',
    button_style='warning',
    tooltip='Stop token stats monitoring'
)

def on_stats_start_click(b):
    filters = {
        'volume24h': {'min': stats_min_volume_input.value},
        'mc': {'min': stats_min_mc_input.value},
        'liquidity': {'min': stats_min_liquidity_input.value}
    }
    token_stats_monitor.start_monitoring(filters)

def on_stats_stop_click(b):
    token_stats_monitor.stop_monitoring()

stats_start_button.on_click(on_stats_start_click)
stats_stop_button.on_click(on_stats_stop_click)

print("Real-time Token Statistics Monitoring:")
display(widgets.VBox([
    widgets.HTML('<b>Filter Settings:</b>'),
    stats_min_volume_input,
    stats_min_mc_input,
    stats_min_liquidity_input,
    widgets.HBox([stats_start_button, stats_stop_button])
]))

print("\nToken Stats Features:")
print("   Real-time price and volume updates")
print("   Market cap and liquidity tracking")
print("   24h price change monitoring")
print("   Customizable filtering by volume, market cap, and liquidity")
print("   Reference: https://docs.birdeye.so/docs/subscribe_token_stats#/")

## Congratulations!

You've successfully built a comprehensive real-time token monitoring system! Here's what you've accomplished:

### What You've Built:
- **Interactive OHLCV Charts**: Professional candlestick charts with custom token support
- **Real-time Price Monitoring**: WebSocket-powered live price feeds for multiple tokens
- **Live Transaction Feeds**: Real-time transaction monitoring with detailed activity tracking
- **Real-time OHLCV Updates**: Live candlestick data with multiple timeframe support
- **Advanced Token Statistics**: Filtered monitoring of market cap, volume, and liquidity

### Challenge Ideas:
- Add real-time alerts and notifications
- Implement custom trading signals
- Create automated monitoring dashboards
- Build multi-token comparison tools
- Add historical data correlation analysis
- Implement risk management monitoring

### Workshop Complete!

Great work! You're now ready to build the next generation of real-time blockchain applications!

### Additional Resources:
- [Birdeye API Documentation](https://docs.birdeye.so/reference)
- [Birdeye WebSocket Documentation](https://docs.birdeye.so/docs/websocket)